<a href="https://colab.research.google.com/github/Amr0093/Amr0093/blob/main/2_Evaluating_language_knowledge_of_ELL_students_from_grades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import the dataset to Kaggle

In [ ]:
import os
user = 'amrahmed93'
key = 'e046f1a9930f5caf3b73e3d4c96317b2'
if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c feedback-prize-english-language-learning -p /content/sample_data/English_Learning_Prize

  0% 0.00/2.80M [00:00<?, ?B/s]
100% 2.80M/2.80M [00:00<00:00, 205MB/s]


In [ ]:
!unzip /content/sample_data/English_Learning_Prize/feedback-prize-english-language-learning.zip -d /content/sample_data/English_Learning_Prize

Archive:  /content/sample_data/English_Learning_Prize/feedback-prize-english-language-learning.zip
  inflating: /content/sample_data/English_Learning_Prize/sample_submission.csv  
  inflating: /content/sample_data/English_Learning_Prize/test.csv  
  inflating: /content/sample_data/English_Learning_Prize/train.csv  


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Loading dependencies

import os
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
pd.set_option('display.max_colwidth',200)

## Read the training and testing data from Pandas

In [ ]:
train_df = pd.read_csv('/content/sample_data/English_Learning_Prize/train.csv')
test_df  =pd.read_csv('/content/sample_data/English_Learning_Prize/test.csv')

In [ ]:
train_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,"I think that students would benefit from learning at home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at th...",3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it do the best on you no matter what is happening it can change your mind. sometimes you need to wake up and look what is around you because problems are...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school policy of having a grade b average that unfair. Because many students have a C average. So that means that they cant go out for sports or other activities...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,"The best time in life is when you become yourself. I agree that the greatest accomplishment, is when you be yourself in a world that constantly trying to make you something else. Because you make ...",4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other people can change people to become better persons you can have an impact of kindess with a homeles that can change his life or with some who needed they a...,2.5,3.0,3.0,3.0,2.5,2.5


In [ ]:
train_df.drop(['text_id'], axis=1, inplace=True)

In [ ]:
train_df.head()

,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,"I think that students would benefit from learning at home,because they wont have to change and get up early in the morning to shower and do there hair. taking only classes helps them because at th...",3.5,3.5,3.0,3.0,4.0,3.0
1,When a problem is a change you have to let it do the best on you no matter what is happening it can change your mind. sometimes you need to wake up and look what is around you because problems are...,2.5,2.5,3.0,2.0,2.0,2.5
2,"Dear, Principal\n\nIf u change the school policy of having a grade b average that unfair. Because many students have a C average. So that means that they cant go out for sports or other activities...",3.0,3.5,3.0,3.0,3.0,2.5
3,"The best time in life is when you become yourself. I agree that the greatest accomplishment, is when you be yourself in a world that constantly trying to make you something else. Because you make ...",4.5,4.5,4.5,4.5,4.0,5.0
4,Small act of kindness can impact in other people can change people to become better persons you can have an impact of kindess with a homeles that can change his life or with some who needed they a...,2.5,3.0,3.0,3.0,2.5,2.5


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3911 entries, 0 to 3910
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   full_text    3911 non-null   object 
 1   cohesion     3911 non-null   float64
 2   syntax       3911 non-null   float64
 3   vocabulary   3911 non-null   float64
 4   phraseology  3911 non-null   float64
 5   grammar      3911 non-null   float64
 6   conventions  3911 non-null   float64
dtypes: float64(6), object(1)
memory usage: 214.0+ KB


In [ ]:
train_df.columns

Index(['full_text', 'cohesion', 'syntax', 'vocabulary', 'phraseology',
       'grammar', 'conventions'],
      dtype='object')

In [ ]:
train_df.describe()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
count,3911.000000,3911.000000,3911.000000,3911.000000,3911.000000,3911.000000
mean,3.127077,3.028254,3.235745,3.116850,3.032856,3.081053
std,0.662542,0.644399,0.583148,0.655997,0.699841,0.671450
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.500000,2.500000,3.000000,2.500000,2.500000,2.500000
50%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,3.500000,3.500000,3.500000,3.500000,3.500000,3.500000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [ ]:
import nltk
#download "stopwords"
nltk.download("stopwords")
from nltk.corpus import stopwords

def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
def label_encode_columns(df, columns):
    encoders = {}
    for col in columns:
        le = LabelEncoder().fit(df[col])
        df[col] = le.transform(df[col])
        encoders[col] = le
    return df, encoders

## Split the training dataset into train and validation data

Prepare the X (input) and the the training target

In [ ]:
X = train_df['full_text']
y = train_df[['cohesion', 'syntax', 'vocabulary', 'phraseology','grammar', 'conventions']]

In [ ]:
y.head()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,3.5,3.5,3.0,3.0,4.0,3.0
1,2.5,2.5,3.0,2.0,2.0,2.5
2,3.0,3.5,3.0,3.0,3.0,2.5
3,4.5,4.5,4.5,4.5,4.0,5.0
4,2.5,3.0,3.0,3.0,2.5,2.5


In [ ]:
# Preprocess text
X_train_preprocessed = [text_preprocessing(text) for text in X]

In [ ]:
#Fit and transform the training dataset, returing both the new training dataset and the fitted encoders to use on the scoring dataset
y_train_preprocessed, encoders = label_encode_columns(df=y, columns=y.columns)

In [ ]:
y_train_preprocessed.head()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,5,5,4,4,6,4
1,3,3,4,2,2,3
2,4,5,4,4,4,3
3,7,7,7,7,6,8
4,3,4,4,4,3,3


In [ ]:
# Defining Train and Validation sets
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val =\
    train_test_split(X_train_preprocessed, y_train_preprocessed, test_size=0.3, random_state=101)

# Method 1 : Using the pipeline with countvectorizer and Multiouput classifier

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pipe_lr = Pipeline(steps=[('cv',CountVectorizer()),
                          ('lr_multi',MultiOutputClassifier(LogisticRegression()))])

In [ ]:
pipe_lr.fit(X_train,y_train) 

Pipeline(steps=[('cv', CountVectorizer()),
                ('lr_multi',
                 MultiOutputClassifier(estimator=LogisticRegression()))])

In [ ]:
pipe_lr.score(X_val,y_val)

0.0008517887563884157

## Method 2: Using the TfidVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Calculate TF-IDF
tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         binary=True,
                         smooth_idf=False)
X_train_tfidf = tf_idf.fit_transform(X_train)
X_val_tfidf = tf_idf.transform(X_val)

In [ ]:
X_train_tfidf

<2737x726347 sparse matrix of type '<class 'numpy.float64'>'
	with 1304835 stored elements in Compressed Sparse Row format>

In [ ]:
model=MultiOutputClassifier(LogisticRegression())
clf = model.fit(X_train_tfidf, y_train)

In [ ]:
print(model.score(X_train_tfidf, y_train))

0.5487760321519912


In [ ]:
y_pred  = model.predict(X_val_tfidf)
print(y_pred )

[[5 4 4 4 4 4]
 [4 4 4 4 4 4]
 [5 4 4 4 4 5]
 ...
 [4 4 4 5 4 4]
 [4 4 4 4 4 4]
 [4 4 4 4 4 4]]


In [ ]:
y_val2=y_val.to_numpy()

In [ ]:
from sklearn.metrics import classification_report

auc_y1 = classification_report(y_val2[:,0],y_pred[:,0])
auc_y2 = classification_report(y_val2[:,1],y_pred[:,1])

print(auc_y1,auc_y2)

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         8
           2       0.00      0.00      0.00        89
           3       0.29      0.02      0.03       225
           4       0.29      0.73      0.42       332
           5       0.32      0.36      0.34       303
           6       0.00      0.00      0.00       156
           7       0.00      0.00      0.00        52
           8       0.00      0.00      0.00         9

    accuracy                           0.30      1174
   macro avg       0.11      0.14      0.10      1174
weighted avg       0.22      0.30      0.21      1174
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00        11
           2       0.00      0.00      0.00       119
           3       0.00      0.00      0.00       259
           4       0.31      0.95      0.47       368
           5       0.21 

In [ ]:
model.score(X_val_tfidf,y_val)

0.004258943781942078